# Float BGC Bias Correction, parallel version

Original version: Veronica Tamsitt (USF)

Current version: Seth Bushinsky, Zachary Nachod (UH Manoa)

Adapted from original MATLAB code written by Seth Bushinsky (UH)

    Download and process GLODAP data
    apply float bias corrections and calculate derivative variables (pH, TALK)
    do float - glodap crossover comparison
    do float - float crossover comparison

Link to MATLAB LIAR/LIPHR code: https://github.com/BRCScienceProducts/LIRs


In [1]:
import xarray as xr
import glob, os
import time
from multiprocessing import Pool
import functions.float_data_processing as fl
import numpy as np
import functions.argo_interp_and_crossover as aiac
import functions.carbon_utils as carbon_utilities
import PyCO2SYS as pyco2


In [4]:
# read in a user-created text file to point to local directories to avoid having to change this every time 
# we update code
lines=[]
with open('path_file.txt') as f:
    lines = f.readlines()
    
count = 0
for line in lines:
    count += 1
    index = line.find("=")
    #print(f'line {count}: {line}')
    #print(index)
    #print(line[0:index])
    line = line.rstrip()
    if line[0:index].find("argo")>=0:
        argo_path=line[index+1:]
    elif line[0:index].find("liar")>=0:
        liar_dir=line[index+1:]
    elif line[0:index].find("matlab")>=0:
        matlab_dir=line[index+1:]



In [9]:
# User definted inputs

adjustment = False # allows read in of outputs from previous crossover and apply them in a rough approximation of correction 

#pressure limits for interpolation of 
p_interp_min = 1450 #minimum pressure for float crossover comparison
p_interp_max = 2000 #maximum pressure for float crossover comparison

# p_interp_min = 500 #minimum pressure for float crossover comparison
# p_interp_max = 2000 #maximum pressure for float crossover comparison
# p_interp_min = 1 #minimum pressure for float crossover comparison
# p_interp_max = 550 #maximum pressure for float crossover comparison
#pressure levels to interpolate to, every 1db
p_interp = np.arange(p_interp_min,p_interp_max+1)

# select glodap pressure range for comparison
# p_compare_min = 400
# p_compare_max = 2100

# p_compare_min = 1
# p_compare_max = 500

p_compare_min = 1400
p_compare_max = 2100

#max density difference to store crossover
# delta_dens = 0.005
delta_dens = 0.05

#max spice difference to store crossover
delta_spice = 0.005
# delta_spice = 0.05

# max pressure difference to store crossover
delta_press = 100
# delta_press = 25

#crossover distance range
# dist = 50
dist = 100



# choose whether to use ESPER or LIPHR for GLODAP crossover comparison
# pH_alg = 'LIPHR'
pH_alg = 'ESPER'


# when making major changes, list version number here
ver_n = '14' 
# v11 - fixing Data Mode issue
# v2 - moving interpolated spice and density calculation to post-PSAL and TEMP interpolation
# v3 - fixed PH_25C calculation for float data, fixed in situ pH comparison (I think)
# v4 - added back in SI and NO3 to DIC calculation - makes a difference apparently (also changes which points have valid data)
# v5 - trying to do near-surface comparisons as well 
# v6 - working on full depth comparison that I will then separate by depth 
# v7 - trying to move code to parallel computing 
# v8 - now can choose whether to use ESPER or LIPHR to calculate GLODAP pH values for comparison 
# v9 - adding correction calculation option for nitrate, pH, dic
# v10 - adding pHCalcTF flag to ESPER calculation - this instructs ESPER to calculate pH for agreement with DIC/TA, not spec pH 
# v12 - changing pHCalcTF back to 0 (false) - consistent with MBARI's processing now 
# v13 - changing pH 25C comparison to be at 1500 db instead of in situ pressure
# v14 - re-doing full depth crossover comparison
run_str = str(dist) + 'km_' \
    + str(p_compare_min) + '_to_' + str(p_compare_max) + '_' + str(delta_press) + 'm_' + \
    str(delta_dens) + 'dens_' + str(delta_spice) + 'spice' + '_' + pH_alg + '_' + ver_n

if not os.path.isdir(argo_path):
    os.mkdir(argo_path)

# Set the paths
if adjustment is True:
    print('Using adjusted path')
    argo_path = argo_path + '../Corrected/Sprof/'

output_dir = argo_path + '../output_' + run_str + '/'
data_dir = 'data/'

#check directories exist
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)
if not os.path.isdir('data'):
    os.mkdir('data')

# Check for a glodap_offsets_plots directory, create if it does not exist
offset_dir = output_dir + 'glodap_offset_plots/'
if not os.path.isdir(offset_dir):
    os.mkdir(offset_dir)


glodap_file_offsets_dir = output_dir + 'glodap_file_offsets_' + run_str + '/'

if not os.path.isdir(glodap_file_offsets_dir):
    os.mkdir(glodap_file_offsets_dir)

argo_path_interpolated = argo_path+'../interpolated_for_crossovers_' + run_str + '/'
if not os.path.isdir(argo_path_interpolated):
    os.mkdir(argo_path_interpolated)

#add derived float file directory within argo_path
argo_path_derived = argo_path+'../derived_for_crossovers_' + run_str + '/'
if not os.path.isdir(argo_path_derived):
    os.mkdir(argo_path_derived)

glodap_offsets_filename = 'glodap_offsets_' + run_str + '.nc'

In [10]:
# Inputs that usually will not change:

#variables to do crossovers
var_list_plot = ['PRES_ADJUSTED','TEMP_ADJUSTED','PSAL_ADJUSTED','DOXY_ADJUSTED','NITRATE_ADJUSTED',
                 'DIC','pH_25C_TOTAL_ADJUSTED','PH_IN_SITU_TOTAL_ADJUSTED','PDENS', 'pH_25C_T_P1500']

qc_data_fields = ['TEMP_ADJUSTED', 'PSAL_ADJUSTED', 'DOXY_ADJUSTED', 'NITRATE_ADJUSTED', 
                  'PRES_ADJUSTED', 'PH_IN_SITU_TOTAL_ADJUSTED']

bgc_data_fields = ['DOXY_ADJUSTED', 'NITRATE_ADJUSTED', 'PH_IN_SITU_TOTAL_ADJUSTED']

#variables to save to derived file
derived_list = ['TEMP_ADJUSTED', 'PSAL_ADJUSTED', 'DOXY_ADJUSTED', 'NITRATE_ADJUSTED', 'PH_IN_SITU_TOTAL_ADJUSTED',
            'pH_25C_TOTAL_ADJUSTED', 'PDENS', 'spice', 'PRES_ADJUSTED', 'DIC','TALK_LIAR', 'pH_25C_T_P1500']

# variables to do interpolation on - must be present here for crossovers to work
interpolation_list = ['TEMP_ADJUSTED', 'PSAL_ADJUSTED', 'DOXY_ADJUSTED', 'NITRATE_ADJUSTED', 'PH_IN_SITU_TOTAL_ADJUSTED',
            'pH_25C_TOTAL_ADJUSTED', 'PRES_ADJUSTED', 'DIC','TALK_LIAR', 'pH_25C_T_P1500']

## 1. Download and process GLODAP data

In [11]:
gdap = fl.get_glodap(data_dir, year = 2023)
gdap.G2longitude[gdap.G2longitude < 0.] = gdap.G2longitude[gdap.G2longitude < 0.] + 360.
#set flagged data to NaN (is this needed? or masked array better?)
# flagvars = ['G2salinity','G2oxygen','G2nitrate','G2tco2','G2talk','G2phts25p0']
flagvars = ['G2salinity','G2oxygen','G2nitrate','G2tco2','G2talk','G2phts25p0', 'G2phtsinsitutp']

for v in flagvars:
    flag = v+'f'
    naninds = gdap[flag]!=2
    gdap[v][naninds] = np.nan

# GLODAP derived variables: density, MLD and pH

#calc potential density
gdap['sigma0_calculated'] = carbon_utilities.sigma0(gdap.G2salinity.values,gdap.G2temperature.values,
                                  gdap.G2longitude.values,gdap.G2latitude.values,gdap.G2pressure.values)
#calculate spice
gdap['spice'] = carbon_utilities.spiciness0(gdap.G2salinity.values,gdap.G2temperature.values,
                                  gdap.G2longitude.values,gdap.G2latitude.values,gdap.G2pressure.values)

#pH from LIPHR
# calculate LIPHR pH at Glodap points below 1480 m and above 2020m (V: where does the depth restriction come in?)
LIPHR_path = liar_dir
Coordinates = np.stack((gdap.G2longitude.values.flatten(), 
                        gdap.G2latitude.values.flatten(), 
                        gdap.G2pressure.values.flatten()),
                        axis=1)
Measurements = np.stack((gdap.G2salinity.values.flatten(), 
                         gdap.G2temperature.values.flatten(), 
                         gdap.G2oxygen.values.flatten()),
                         axis=1)

if pH_alg=='LIPHR':           
    MeasIDVec = [1, 7, 6]
    results = carbon_utilities.LIPHR_matlab(LIPHR_path,
                                        Coordinates.tolist(),
                                        Measurements.tolist(),
                                        MeasIDVec, 
                                        OAAdjustTF = False)            
elif pH_alg=='ESPER':
    MeasIDVec_ESPER = [1, 2, 6] # S, T, O2 - different numbering than v2 LIRs
    Equations = 7 # for ESPER - asking to use equation w/ S, T, and O2 only 
    DesiredVariables = [3] # in situ pH on total scale 

    # calculate decimal_year for ESPER
    da = gdap.datetime
    decimal_year = da.dt.year + (da.dt.dayofyear - 1 + (da.dt.hour * 3600 + da.dt.minute * 60 + da.dt.second) / 86400) / (365 + da.dt.is_leap_year)
    results = carbon_utilities.ESPER_mixed_matlab(LIPHR_path,
                                                    DesiredVariables,
                                                    Coordinates.tolist(),
                                                    Measurements.tolist(),
                                                    MeasIDVec_ESPER,
                                                    Equations, 
                                                    decimal_year.values.tolist(), 
                                                    0, 0)
gdap['pH_in_situ_total'] = results
gdap.pH_in_situ_total[np.isnan(gdap.G2phts25p0)] = np.nan
# gdap pH 25C 


results = pyco2.sys(
    par1=2300, 
    par2=gdap.pH_in_situ_total,
    par1_type=1,
    par2_type=3,
    temperature=gdap.G2temperature, 
    pressure=gdap.G2pressure, 
    salinity=gdap.G2salinity, 
    temperature_out=25., #fixed 25C temperature
    pressure_out=1500., # fixed output pressure
    opt_pH_scale = 1, #total
    opt_k_carbonic=10, #Lueker et al. 2000
    opt_k_bisulfate=1, # Dickson 1990 (Note, matlab co2sys combines KSO4 with TB. option 3 = KSO4 of Dickson & TB of Lee 2010)
    opt_total_borate=2, # Lee et al. 2010
    opt_k_fluoride=2, # Perez and Fraga 1987
    opt_buffers_mode=1, # used to be "buffers_mode='auto'" but seems to have changed in versions of pyco2?
)

pH_25Cp1500 = results['pH_total_out']
gdap['pH_25C_T_P1500'] = pH_25Cp1500
gdap.pH_25C_T_P1500[np.isnan(gdap.G2phts25p0)]=np.nan

gdap['pH_25C_TOTAL_ADJUSTED'] = carbon_utilities.co2sys_pH25C(2300.,gdap.pH_in_situ_total,gdap.G2temperature,
                                                         gdap.G2salinity,gdap.G2pressure)
#set pH to nan where there was no original pH data from GLODAP
gdap.pH_25C_TOTAL_ADJUSTED[np.isnan(gdap.G2phts25p0)]=np.nan

#rename GLODAP comparison variables to match argo
gdap = gdap.rename(columns={'G2longitude':'LONGITUDE', 'G2latitude':'LATITUDE', 'G2pressure':'PRES_ADJUSTED',
                            'G2temperature':'TEMP_ADJUSTED','G2salinity':'PSAL_ADJUSTED', 
                            'G2oxygen':'DOXY_ADJUSTED','G2nitrate':'NITRATE_ADJUSTED', 'G2tco2':'DIC', 
                            'G2talk':'TALK_LIAR', 'G2MLD':'MLD','G2o2sat':'o2sat', 'G2PTMP':'PTMP', 
                            'pH_in_situ_total':'PH_IN_SITU_TOTAL_ADJUSTED','sigma0_calculated':'PDENS'})

gdap['obs_index']=gdap.reset_index().index

https://www.ncei.noaa.gov/data/oceans/ncei/ocads/data/0283442/GLODAPv2.2023_Merged_Master_File.csv


--2024-10-22 10:52:46--  https://www.ncei.noaa.gov/data/oceans/ncei/ocads/data/0283442/GLODAPv2.2023_Merged_Master_File.csv
Resolving www.ncei.noaa.gov (www.ncei.noaa.gov)... 205.167.25.168, 205.167.25.178, 205.167.25.172, ...
Connecting to www.ncei.noaa.gov (www.ncei.noaa.gov)|205.167.25.168|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 894645727 (853M) [text/csv]
Saving to: ‘data/GLODAPv2.2023_Merged_Master_File.csv’

     0K .......... .......... .......... .......... ..........  0%  127K 1h54m
    50K .......... .......... .......... .......... ..........  0%  256K 85m48s
   100K .......... .......... .......... .......... ..........  0% 74.5M 57m15s
   150K .......... .......... .......... .......... ..........  0%  256K 57m10s
   200K .......... .......... .......... .......... ..........  0% 31.1M 45m49s
   250K .......... .......... .......... .......... ..........  0% 56.8M 38m13s
   300K .......... .......... .......... .......... ..........  0%  1

/var/folders/7r/4f_w_nb56llcwh96t_pjq_qh0000gn/T/ipykernel_6013/3136098330.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdap.pH_in_situ_total[np.isnan(gdap.G2phts25p0)] = np.nan
/Users/smb-uh/opt/anaconda3/envs/float_oxygen_offset_impacts/lib/python3.9/site-packages/autograd/tracer.py:48: RuntimeWarning: invalid value encountered in log
  return f_raw(*args, **kwargs)
/Users/smb-uh/opt/anaconda3/envs/float_oxygen_offset_impacts/lib/python3.9/site-packages/PyCO2SYS/equilibria/p1atm.py:419: RuntimeWarning: overflow encountered in power
  K1 = 10.0**-pK1  # this is on the NBS scale
/Users/smb-uh/opt/anaconda3/envs/float_oxygen_offset_impacts/lib/python3.9/site-packages/PyCO2SYS/equilibria/p1atm.py:479: RuntimeWarning: overflow encountered in power
  K1 = 10.0**-pK1  # this is on the NBS scale
/Use

In [12]:
from importlib import reload

#toggle to plot offsets profile by profile
plot_profile = 0

reload(aiac)

start_time = time.perf_counter()

# 0: overwrites and runs all floats in the argo_path directory 
# 1: checks all argo files against floats with a derived file already created. Only runs floats if no derived file  
# 2: runs specific floats listed below
append_data = 0
num_processes = 18  

if 'argo_interp' in locals():
    argo_interp.close()

if 'argolist_run' in locals():
    del argolist_run

argolist = []

for file in os.listdir(argo_path):
    if file.endswith('Sprof.nc'):
        argolist.append(file)
argolist.sort()

if append_data==1: # only run on files that do not have a derived file created yet
    derivedfiles = []

    for file in os.listdir(argo_path_derived):
        if file.endswith('derived.nc'):
            derivedfiles.append(file[0:7])
    derivedfiles.sort()

    if 'argolist_run' in locals():
        del argolist_run

    # list of all files not in the derived file list
    argolist_run = []
    print(len(argolist))
    for file in argolist:
        # print(file)
        if file[0:7] not in derivedfiles:
            argolist_run.append(file)
    print(len(argolist_run))

elif append_data==0:
    argolist_run=argolist
   
else:
    # argolist_run = ['5906547_Sprof.nc']
    argolist_run = ['5904166_Sprof.nc']
    # argolist_run = ['5906547_Sprof.nc',
    #                     '5906548_Sprof.nc',
    #                     '5906549_Sprof.nc', 
    #                     '5906550_Sprof.nc', 
    #                     '5906551_Sprof.nc', 
    #                     '5906552_Sprof.nc', 
    #                     '5906553_Sprof.nc',
    #                     '5906562_Sprof.nc',
    #                     '5906554_Sprof.nc',
    #                     '5906561_Sprof.nc', 
    #                     '5906556_Sprof.nc',
    #                     '5906558_Sprof.nc',
    #                     '5906559_Sprof.nc',
    #                     '5906557_Sprof.nc']

#restrict glodap data to comparison pressure range
gdap_p = gdap[(gdap.PRES_ADJUSTED.values>p_compare_min) & (gdap.PRES_ADJUSTED.values<p_compare_max)]


if __name__ == "__main__":
    
    with Pool(processes=num_processes) as pool:
        # Create a list of arguments for pool.starmap
        argo_args = [(argo_path, liar_dir, argo_path_interpolated, argo_path_derived, file, qc_data_fields, bgc_data_fields, p_interp, derived_list, interpolation_list, adjustment) for file in argolist_run]
        
        # Use pool.starmap with the list of arguments
        pool.starmap(aiac.argo_interp_profiles, argo_args)
    
    
    # only run glodap crossovers on floats that have an interpolated file 
    with Pool(processes=num_processes) as pool:
        # Create a list of arguments for pool.starmap
        argo_args = [(argo_path_interpolated, offset_dir, glodap_file_offsets_dir, file, dist, delta_dens, delta_spice, delta_press, \
                        gdap_p, p_interp, plot_profile, var_list_plot, p_compare_min, p_compare_max) for file in argolist_run]
        
        # Use pool.starmap with the list of arguments
        pool.starmap(aiac.glodap_crossover_offsets, argo_args)

# Now load in all individual offset files and concatenate into larger file
crossover_list = []
for file in os.listdir(glodap_file_offsets_dir):
    if file.endswith('_offset.nc'):
        crossover_list.append(file)
# print(len(crossover_list))

if 'glodap_offsets' in locals():
       del glodap_offsets # deletes argo_interp in case this code is being run multiple times. 

for idx, gdap_offset_file in enumerate(crossover_list):
    # print(idx)
    # print(gdap_offset_file)
    gdap_offset_n = xr.open_dataset(glodap_file_offsets_dir + gdap_offset_file)

    if len(gdap_offset_n['N_CROSSOVERS'])>0:
        if 'glodap_offsets' not in locals(): # modified to deal w/ situation where n==0 skipped defining argo_interp
            glodap_offsets = gdap_offset_n
        else:
            glodap_offsets = xr.concat([glodap_offsets,gdap_offset_n],'N_CROSSOVERS')
print(glodap_offsets)

glodap_offsets.to_netcdf(output_dir+glodap_offsets_filename)

print('Total number of glodap crossovers: ' + str(len(glodap_offsets.N_CROSSOVERS)))

finish_time = time.perf_counter()
print("Program finished in {} seconds - using multiprocessing".format(finish_time - start_time))
print("---")

NameError: name 'glodap_offsets' is not defined

In [14]:
argo_path_derived

'/Users/smb-uh/UHM_Ocean_BGC_Group Dropbox/Datasets/Data_Products/BGC_ARGO_GLOBAL/2024_10_22_GBC_R1/Sprof/../derived_for_crossovers_100km_1400_to_2100_100m_0.05dens_0.005spice_ESPER_14/'